---
# Core implementation of Linked List

In [1]:
def _standardize_start_stop(start, stop, size):
    """Convenience function for standardizing start, stop indexes
    for slicing functionality.
    
    It pushes start and stop into the range [0, size].
     1. Allows for negative indexing, -i -> size-i
     2. Replaces default None from slice() with expected start/stop values.
    """
    try:
        if start is None:
            start = 0
        elif start < 0:
            start += size

        if stop is None:
            # Note: This means for slicing, None gives the exclusive end of slice.
            #       Therefore, obj[i:] -> [obj[i], obj[i+1], ..., obj[-1]] correctly.
            stop = size
        elif stop < 0:
            stop += size
    except TypeError:
        raise TypeError("Start and stop index must be integer.")

    return start, stop


In [2]:
class _BaseLinkedList:
    """This is implementation of a forward, singlely-linked list.
    
    Attributes:
      ._head
      ._size
    
    Methods:
      ._load_list(self, start, stop)
      ._iternodes(start=None, stop=None)
      ._get_node(idx)
      
    Notes:
    
    Items packed in via nodes expressed as node=[<item>, <next_node>].  So it's a deeply nested structure, i.e.
    >>> LinkedList(1, 2, 3)._head == [1, [2, [3, None]]]
    True
    
    Carries information about the size of the linked list as a member variable:
    >>> LinkedList(1, 2, 3)._size
    3
    """
    
    def __init__(self, *items):
        self._load_list(*items)
    
    def _load_list(self, *items):
        """Loads items into nested [a_0, [a_1, ..., [a_n-1, None]]] structure."""
        self._size = len(items)
        
        if not items:
            self._head = []
        else:
            self._head = [items[0], None]
            i = 1
            node = self._head
            while i < self._size:
                new_node = [items[i], None]
                node[1] = new_node
                node = new_node
                i += 1
        
    def _iternodes(self, start=None, stop=None):
        """Iterates through the nodes, returning each node at a time.
        
        Parameters:
         * start : [None, int]
           Start with node at index=start.
         * stop : [None, int]
           Stop yielding nodes at index=stop (i.e. exclusive end).
        """
        start, stop = _standardize_start_stop(start, stop, self._size)
        
        if start == 0 and stop == self._size:
            node = self._head
            while node:
                yield node
                node = node[1]
        else:
            node = self._head
            i = 0
            while node:
                if start <= i < stop:
                    yield node
                elif i == stop:
                    break
                node = node[1]
                i += 1

    def _get_node(self, idx):
        if idx < 0:
            idx += self._size
        
        node = self._head
        i = 0
        while node:
            if i == idx:
                return node
            
            node = node[1]
            i += 1
        
        # If nothing returned, it means index is out of bounds.
        raise IndexError("Index out of bounds.")


### Mock-ups for tests:

In [3]:
x = _BaseLinkedList(1, 2, 3)

x._get_node(0)

[1, [2, [3, None]]]

In [4]:
for node in x._iternodes():
    print(node)

[1, [2, [3, None]]]
[2, [3, None]]
[3, None]


In [5]:
try:
    x._get_node(-4)
    raise Exception("Failure: IndexError not called!")
except IndexError as e:
    print(f"Succeeded, IndexError('{e}') raised on index -4.")
    
try:
    x._get_node(4)
    raise Exception("Failure: IndexError not called!")
except IndexError as e:
    print(f"Succeeded, IndexError('{e}') raised on index 4.")

Succeeded, IndexError('Index out of bounds.') raised on index -4.
Succeeded, IndexError('Index out of bounds.') raised on index 4.


---
# Linked List in Python

In [6]:
class LinkedList(_BaseLinkedList):
    """This provides a linked list container for data."""
    def __init__(self, *items):
        self._load_list(*items)
    
    def __len__(self):
        return self._size
            
    def __iter__(self):
        for (item, _) in self._iternodes():
            yield item
    
    def __contains__(self, val):
        for (item, _) in self._iternodes():
            if item == val:
                return True
        return False
    
    def __getitem__(self, key):
        if isinstance(key, int):
            item, _ = self._get_node(key)
            return item
        elif isinstance(key, slice):
            if key.step != None:
                raise ValueError("Slicing with steps not implemented for LinkedList.")
            
            start, stop = _standardize_start_stop(key.start, key.stop, self._size)
            items = []
            for (item, _) in self._iternodes(start, stop):
                items.append(item)
            return items
        else:
            raise TypeError("Index must be integer or slice.")

    def __repr__(self):
        str_of_items = ", ".join((str(item) for item in self))
        return f"LinkedList({str_of_items})"
    
    def append(self, new_item):
        node = self._get_node(-1)
        node[1] = [new_item, None]
        self._size += 1
        
    def prepend(self, new_item):
        self._head = [new_item, self._head]
        self._size += 1
        
    def insert(self, idx, new_item):
        node = self._get_node(idx)
        next_node = node[1]
        node[1] = [new_item, next_node]
        self._size += 1


In [7]:
xlist = LinkedList(1,2,4)
xlist

LinkedList(1, 2, 4)

In [8]:
xlist.prepend(0)
xlist

LinkedList(0, 1, 2, 4)

In [9]:
xlist.append(5)
xlist

LinkedList(0, 1, 2, 4, 5)

In [10]:
xlist.insert(idx=2, new_item=3)
xlist

LinkedList(0, 1, 2, 3, 4, 5)

In [11]:
for item in xlist:
    print(item)

0
1
2
3
4
5


In [12]:
print(f"First item: {xlist[0]}, last item: {xlist[-1]}")

First item: 0, last item: 5


In [13]:
1
2
print(f" xlist[1:3] = {xlist[1:3]}\n xlist[:2] = {xlist[:2]}\n xlist[2:] = {xlist[2:]}\n xlist[:] = {xlist[:]}")

 xlist[1:3] = [1, 2]
 xlist[:2] = [0, 1]
 xlist[2:] = [2, 3, 4, 5]
 xlist[:] = [0, 1, 2, 3, 4, 5]


In [14]:
1 in xlist

True

---
# DoublyLinkedList Implementation

In [15]:
class _BaseDoublyLinkedList:
    """This is implementation of a forward, singlely-linked list.
    
    Attributes:
      ._head
      ._size
    
    Methods:
      ._load_list(self, start, stop)
      ._iternodes(start=None, stop=None)
      ._get_node(idx)
      
    Notes:
    
    Items packed in via nodes expressed as node=[<item>, <next_node>].  So it's a deeply nested structure, i.e.
    >>> LinkedList(1, 2, 3)._head == [1, [2, [3, None]]]
    True
    
    Carries information about the size of the linked list as a member variable:
    >>> LinkedList(1, 2, 3)._size
    3
    """
    
    def __init__(self, *items):
        self._load_list(*items)
    
    def _load_list(self, *items):
        """Loads items into nested [a_0, [a_1, ..., [a_n-1, None]]] structure."""
        
        self._size = len(items)
    
        if not items:
            self._head = []
            self._tail = []
        else:
            self._head = [items[0], None, None]
            i = 1
            node = self._head
            while i < self._size:
                new_node = [items[i], None, node]
                node[1] = new_node
                node = new_node
                i += 1
            self._tail = node

    def _iternodes(self, start=None, stop=None):
        """Iterates through the nodes, returning each node at a time.
        
        Parameters:
         * start : [None, int]
           Start with node at index=start.
         * stop : [None, int]
           Stop yielding nodes at index=stop (i.e. exclusive end).
        """
        
        start, stop = _standardize_start_stop(start, stop, self._size)
        # Just loop through if full list if full range requested.
        if start == 0 and stop == self._size:
            node = self._head
            prev_node = []
            while node:
                yield node
                node = node[1]
        # Otherwise, narrow yields down to requested range.
        else:
            node = self._head
            i = 0
            while node:
                if start <= i < stop:
                    yield node
                elif i == stop:
                    break
                node = node[1]
                i += 1

    def _reversed_iternodes(self, start=None, stop=None):
        """Iterates through the nodes, returning each node at a time.
        
        Parameters:
         * start : [None, int]
           Start with node at index=start.
         * stop : [None, int]
           Stop yielding nodes at index=stop (i.e. exclusive end).
        """

        start, stop = _standardize_start_stop(start, stop, self._size)
        
        # Just loop through if full list if full range requested.
        if start == 0 and stop == self._size:
            node = self._tail
            while node:
                yield node
                node = node[2]
        # Otherwise, narrow yields down to requested range.
        else:
            node = self._tail
            i = self._size-1
            while node:
                if start <= i < stop:
                    yield node
                elif i == start:
                    break
                node = node[2]
                i -= 1

    def _get_node(self, idx):
        if idx < 0:
            idx += self._size
        
        node = self._head
        i = 0
        while node:
            if i == idx:
                return node
            
            node = node[1]
            i += 1
        
        # If nothing returned, it means index is out of bounds.
        raise IndexError("Index out of bounds.")


In [16]:
class DoublyLinkedList(_BaseDoublyLinkedList):
    """This provides a doubly-linked list container for data."""
    
    def __init__(self, *items):
        self._load_list(*items)
    
    def __len__(self):
        return self._size
            
    def __iter__(self):
        for (item, _, _) in self._iternodes():
            yield item
            
    def __reversed__(self):
        for (item, _, _) in self._reversed_iternodes():
            yield item

    def __contains__(self, val):
        for (item, _, _) in self._iternodes():
            if item == val:
                return True
        return False

    def __getitem__(self, key):
        if isinstance(key, int):
            item, _, _ = self._get_node(key)
            return item
        elif isinstance(key, slice):
            if key.step != None:
                raise ValueError("Slicing with steps not implemented for LinkedList.")

            start, stop = _standardize_start_stop(key.start, key.stop, self._size)
            items = []
            for (item, _, _) in self._iternodes(start, stop):
                items.append(item)
            return items
        else:
            raise TypeError("Index must be integer or slice.")

    def __repr__(self):
        str_of_items = ", ".join((str(item) for item in self))
        return f"DoublyLinkedList({str_of_items})"

    def append(self, new_item):
        node = [new_item, None, self._tail]
        self._tail[1] = node 
        self._tail = node

        self._size += 1

    def prepend(self, new_item):
        node = [new_item, self._head, None]
        self._head[2] = node
        self._head = node
        
        self._size += 1

    def insert(self, idx, new_item):
        node = self._get_node(idx)
        prev_node = node[2]
        new_node = [new_item, node, prev_node]
        prev_node[1] = new_node
        node[2] = new_node

        self._size += 1


### Mock-up for tests:

In [17]:
xlist = DoublyLinkedList(1, 2, 3, 4)

for item in xlist:
    print(item)
print()

for item in reversed(xlist):
    print(item)

1
2
3
4

4
3
2
1


In [18]:
xlist[2]

3

In [19]:
2 in xlist

True

In [20]:
xlist.prepend(0)
xlist

DoublyLinkedList(0, 1, 2, 3, 4)

In [21]:
xlist.append(5)
xlist

DoublyLinkedList(0, 1, 2, 3, 4, 5)